In [25]:
import pythonradex

# # import necessary modules
from pythonradex import radiative_transfer, helpers, LAMDA_file, atomic_transition
from scipy import constants
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

In [5]:
# pyhonradex needs a file containing the atomic data, import it, this is a specific file for CO from LAMBDA database
datafilepath = r"C:\Users\alios\OneDrive - University College London\Desktop\UCL\Year3\group project\codes\co_datafile.dat"  # file downloaded from LAMDA database


# define the geometry of the nebula
geometry = "uniform sphere"

# define the parameters of the nebula
line_profile_type = "Gaussian"  # line profile, can be "Gaussian",  “rectangular”,"LVG sphere", "LVG slab”
width_v = 5 * constants.kilo  # line width in m/s, costants.kilo is 1 km/s

cloud = radiative_transfer.Cloud(
    datafilepath=datafilepath,
    geometry=geometry,
    line_profile_type=line_profile_type,
    width_v=width_v,
)

N = 1e18 / constants.centi**2  # CO column density in m-2
Tkin = 30  # kinetic temperature in [K]

# collider densities in cm-3:
para_h2_density = 1e8
ortho_h2_density = 3e8

# collider densities in m-3 (phytonradex takes the densities in m-3):
collider_densities_LTE = {
    "para-H2": para_h2_density / constants.centi**3,
    "ortho-H2": ortho_h2_density / constants.centi**3,
}

# define the background radiation field, in this case we assume that the background is zero, z is the redshift
ext_background = helpers.generate_CMB_background(z=0)

# no dust:
T_dust = 0
tau_dust = 0

# use high colliders density to simulate LTE
cloud.update_parameters(
    N=N,
    Tkin=Tkin,
    collider_densities=collider_densities_LTE,
    ext_background=ext_background,
    T_dust=T_dust,
    tau_dust=tau_dust,
)

# solve the radiative trasfer equation, i.e. calculate the level population with an iterative method
cloud.solve_radiative_transfer()

In [ ]:
# store the data
data_co = LAMDA_file.read(datafilepath, read_frequencies=True)

# Initialize g_weigth
levels = data_co["levels"]
rad_transitions = data_co["radiative transitions"]
coll_transitions = data_co["collisional transitions"]
quantum_numbers = data_co["quantum numbers"]

g_weigth = np.array([level.g for level in levels])
g_up = np.array([gup.name for gup in rad_transitions])

In [21]:
number_level = np.array([level.number for level in levels])

In [22]:
number_level

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40])

In [32]:
rad_transitions

In [43]:
Eu = np.array([level.E for level in levels]) / constants.k

Eu_test = np.array([level.up.E for level in rad_transitions]) / constants.k

In [45]:
Eu

array([   0.        ,    5.53214517,   16.59617609,   33.19188137,
         55.31833031,   82.974736  ,  116.16031175,  154.87340764,
        199.11280441,  248.87685379,  304.16332873,  364.97043585,
        431.29580629,  503.13678297,  580.49056589,  663.35449669,
        751.72520052,  845.599158  ,  944.97313597, 1049.84303913,
       1160.20491752, 1276.0546769 , 1397.3877882 , 1524.19972692,
       1656.48539072, 1794.2399659 , 1937.45806361, 2086.1342947 ,
       2240.26298185, 2399.83801775, 2564.8534387 , 2735.3029919 ,
       2911.18013782, 3092.47804981, 3279.18990371, 3471.30844029,
       3668.82625666, 3871.73609478, 4080.02983772, 4293.69964515,
       4512.73740391])

In [44]:
Eu_test

array([   5.53214517,   16.59617609,   33.19188137,   55.31833031,
         82.974736  ,  116.16031175,  154.87340764,  199.11280441,
        248.87685379,  304.16332873,  364.97043585,  431.29580629,
        503.13678297,  580.49056589,  663.35449669,  751.72520052,
        845.599158  ,  944.97313597, 1049.84303913, 1160.20491752,
       1276.0546769 , 1397.3877882 , 1524.19972692, 1656.48539072,
       1794.2399659 , 1937.45806361, 2086.1342947 , 2240.26298185,
       2399.83801775, 2564.8534387 , 2735.3029919 , 2911.18013782,
       3092.47804981, 3279.18990371, 3471.30844029, 3668.82625666,
       3871.73609478, 4080.02983772, 4293.69964515, 4512.73740391])

In [ ]:
for i, tran in enumerate(rad_transitions):
    gu = tran.up.g
    Eu = tran.up.E
    print(f"Transition {i}:")
    print("Statistical weight of upper level ", gu)
    print("Energy of upper level Eu", Eu / constants.k)
    print()

print("statsitical weight of upper level ", gu)
print("Energy of upper level Eu", Eu / constants.k)

statsitical weight of upper level  3.0
Energy of upper level Eu 5.532145167854433


In [31]:
rad_transitions.up.g

AttributeError: 'list' object has no attribute 'up'

In [ ]:
EmittingMolecule

In [27]:
eup_rad_transition = np.array([atomic_transition.Level.up for transition in rad_transitions])

AttributeError: type object 'Level' has no attribute 'up'

In [19]:
g_up[0]

np.str_('1-0')

In [ ]:
def extract_upper_number(np_str):
    return int(np_str.split("-")[0])


# Example usage
first_number = extract_upper_number(g_up[0])

print(first_number)  # Output: 1

1


In [8]:
g_weigth

array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19., 21., 23., 25.,
       27., 29., 31., 33., 35., 37., 39., 41., 43., 45., 47., 49., 51.,
       53., 55., 57., 59., 61., 63., 65., 67., 69., 71., 73., 75., 77.,
       79., 81.])

# Testing methanol 

In [ ]:
# pyhonradex needs a file containing the atomic data, import it, this is a specific file for CO from LAMBDA database
datafilepath = (
    r"C:\Users\alios\OneDrive - University College London\Desktop\UCL\Year3\group project\codes\e_ch3oh.dat"  # file downloaded from LAMDA database
)


# define the geometry of the nebula
geometry = "uniform sphere"

# define the parameters of the nebula
line_profile_type = "Gaussian"  # line profile, can be "Gaussian",  “rectangular”,"LVG sphere", "LVG slab”
width_v = 5 * constants.kilo  # line width in m/s, costants.kilo is 1 km/s

cloud = radiative_transfer.Cloud(
    datafilepath=datafilepath,
    geometry=geometry,
    line_profile_type=line_profile_type,
    width_v=width_v,
)

N = 1e18 / constants.centi**2  # CO column density in m-2
Tkin = 30  # kinetic temperature in [K]

# collider densities in cm-3:
para_h2_density = 1e8
ortho_h2_density = 3e8

# collider densities in m-3 (phytonradex takes the densities in m-3):
LTE_collider_densities_h2only = {"H2": 1e8 / constants.centi**3}

# define the background radiation field, in this case we assume that the background is zero, z is the redshift
ext_background = helpers.generate_CMB_background(z=0)

# no dust:
T_dust = 0
tau_dust = 0

# use high colliders density to simulate LTE
cloud.update_parameters(
    N=N,
    Tkin=Tkin,
    collider_densities=LTE_collider_densities_h2only,
    ext_background=ext_background,
    T_dust=T_dust,
    tau_dust=tau_dust,
)

# solve the radiative trasfer equation, i.e. calculate the level population with an iterative method
cloud.solve_radiative_transfer()

79-78: tau_nu0 = -0.0215
121-120: tau_nu0 = -0.0107
137-136: tau_nu0 = -4.31e-06


c:\Users\alios\AppData\Local\Programs\Python\Python312\Lib\site-packages\pythonradex\radiative_transfer.py:333: UserWarning: negative optical depth!
  warnings.warn('negative optical depth!')


In [ ]:
# store the data
data = LAMDA_file.read(datafilepath, read_frequencies=False)

# Initialize g_weigth
levels = data["levels"]
rad_transitions = data["radiative transitions"]
coll_transitions = data["collisional transitions"]
quantum_numbers = data["quantum numbers"]


In [54]:
Eu_test = np.array([transition.up.E for transition in rad_transitions]) / constants.k

In [51]:
Eu_test

array([ 175.52646272,   36.33487126,  215.89566435, ...,  788.06989178,
        972.79157753, 1405.59868271])

In [55]:
gup = np.array([level.up.g for level in rad_transitions])
glow = np.array([level.low.g for level in rad_transitions])


In [59]:
print(gup[:10])

[23.  9. 23. 13. 19.  5. 29. 15. 17.  5.]


In [60]:
print(glow[:10])

[21.  7. 21. 15. 17.  7. 31. 13. 19.  7.]
